<a href="https://colab.research.google.com/github/dakim01/My-Data-projects/blob/master/TrustPilot_Reviews_o_TikTok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

from bs4 import BeautifulSoup
import requests
import pandas as pd
import datetime as dt
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

In [ ]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install transformers pandas numpy

Looking in indexes: https://download.pytorch.org/whl/cu121


In [ ]:
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

In [ ]:
tokens = tokenizer.encode('This is amazing, I love it. great!', return_tensors = 'pt')
tokens

tensor([[  101, 10372, 10127, 39854,   117,   151, 11157, 10197,   119, 11838,
           106,   102]])

In [ ]:
tokenizer.decode(tokens[0])

'[CLS] this is amazing, i love it. great! [SEP]'

In [ ]:
result = model(tokens)
result

SequenceClassifierOutput(loss=None, logits=tensor([[-2.4825, -2.9365, -1.5084,  1.3381,  4.5352]],
       grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [ ]:
int(torch.argmax(result.logits))+1

5

In [ ]:
# Initialize lists
review_titles = []
review_dates_original = []
review_dates = []
review_ratings = []
review_texts = []
page_number = []

In [ ]:
# Set Trustpilot page numbers to scrape here
from_page = 1
to_page = 48

In [ ]:
for i in range(from_page, to_page + 1):
    response = requests.get(f"https://www.trustpilot.com/review/tiktok.com?page={i}")
    web_page = response.text
    soup = BeautifulSoup(web_page, "html.parser")

    for review in soup.find_all(class_ = "styles_reviewCardInner__EwDq2"):
        # Review titles
        review_title = review.find(class_ = "typography_heading-s__f7029 typography_appearance-default__AAY17")
        review_titles.append(review_title.getText())
        # Review dates
        review_date_original = review.select_one(selector="time")
        # print(review_date_original)
        review_dates_original.append(review_date_original.get('datetime'))

        # Convert review date texts into Python datetime objects
        review_date = review.select_one(selector="time").getText().replace("Updated ", "")
        if "hours ago" in review_date.lower() or "hour ago" in review_date.lower():
            review_date = dt.datetime.now().date()
        elif "a day ago" in review_date.lower():
            review_date = dt.datetime.now().date() - dt.timedelta(days=1)
        elif "days ago" in review_date.lower():
            review_date = dt.datetime.now().date() - dt.timedelta(days=int(review_date[0]))
        else:
            review_date = dt.datetime.strptime(review_date, "%b %d, %Y").date()
        review_dates.append(review_date)

         # Review ratings
        review_rating = review.find(class_ = "star-rating_starRating__4rrcf star-rating_medium__iN6Ty").findChild()
        # review_rating = review.find(class_ = "typography_body-l__KUYFJ typography_appearance-subtle__8_H2l").get_text()
        review_ratings.append(review_rating["alt"])
        # review_ratings.append(review_rating)

        # When there is no review text, append "" instead of skipping so that data remains in sequence with other review data e.g. review_title
        review_text = review.find(class_ = "typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn")
        if review_text == None:
            review_texts.append("")
        else:
            review_texts.append(review_text.getText())
            # Trustpilot page number
        page_number.append(i)


In [ ]:
review_text.text

'TikTok is really educational and soothing for music lovers in my opinion. Most of my friends use this app, and I really like that TikTok has a chat feature too so you could share videos with your friends. Lots of social media platforms are toxic, not excluding TikTok, but so far this app is one of the great and less toxic (if u stay out of controversial topics) in my opinion.'

In [ ]:
Review_Data = pd.DataFrame({'Review titles':review_titles,
                            'Review Original Dates':review_dates_original,
                            'Review Dates':review_dates,
                            'Review Ratings':review_ratings,
                            'ReviewTexts':review_texts,
                            'Page Number':page_number})

Review_Data = Review_Data.replace('\n','',regex=True)
Review_Data.head()

,Review titles,Review Original Dates,Review Dates,Review Ratings,ReviewTexts,Page Number
0,Child safeguarding concerns,2023-10-05T01:17:49.000Z,2023-10-05,Rated 1 out of 5 stars,I am aware of a very young child on TikTok. I ...,1
1,Constantly having warnings for saying…,2023-09-28T13:17:33.000Z,2023-09-28,Rated 1 out of 5 stars,Constantly having warnings for saying somethin...,1
2,All went well for first few months then…,2023-09-29T02:21:40.000Z,2023-09-29,Rated 2 out of 5 stars,All went well for first few months then the we...,1
3,This social media outlet is utter…,2023-09-17T21:23:38.000Z,2023-09-17,Rated 1 out of 5 stars,This WOKE social media outlet is utter nonsens...,1
4,Woke social engineering over truth,2023-09-16T13:44:23.000Z,2023-09-16,Rated 1 out of 5 stars,Discrimination of accounts that do not serve t...,1


In [ ]:
def sentiment_score(review_texts):
  tokens2 = tokenizer.encode(ReviewTexts, return_tensors = 'pt')
  result = model(tokens2)
  return int(torch.argmax(result.logits))+1

In [ ]:
Review_Data.to_excel("Review.xlsx")

In [ ]:
from google.colab import drive
drive.mount('/content/drive')